# 1. Setup

## 1.1 Module import

Import necessary modules

In [ ]:
import tensorflow as tf
import keras
import dataset_utils

## 1.2 Dataset import using opencv

Define parameters for the image import

In [ ]:
img_width = 100
img_height = 40

train_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/training_dataset/all_cropped'
validation_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/validation_dataset/all_cropped'

Import the data for the training dataset

In [ ]:
train_labels, train_images = dataset_utils.load_dataset(path_to_dataset = train_dataset_path, change_contrast = False, image_size = (img_width, img_height))

Import the data for the validation dataset

In [ ]:
validation_labels, validation_images = dataset_utils.load_dataset(path_to_dataset = validation_dataset_path, change_contrast = False, image_size = (img_width, img_height))

# 2. Transfer learning

## 2.1 Construct the new model

Load the base model

In [ ]:
IMG_SIZE = (img_height, img_width)
IMG_SHAPE = IMG_SIZE + (3,)

base_model = tf.keras.applications.MobileNetV3Small(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

Add the custom classification layer to it

In [ ]:
# new layers
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer_1 = tf.keras.layers.Dense(32, activation='elu')
prediction_layer = tf.keras.layers.Dense(3, activation = 'softmax')


# construct the final model
inputs = keras.Input(shape = (img_height, img_width, 3))
x = keras.applications.mobilenet_v3.preprocess_input(inputs)
x = base_model(x, training = False)
x = global_average_layer(x)
x = dense_layer_1(x)
#x = keras.layers.Dropout(0.05)(x)
outputs = prediction_layer(x)
model = keras.Model(inputs, outputs)

model.summary()

## 2.2 Start the initial learning process

Define Parameters for the learning process

In [ ]:
base_learning_rate = 0.03
initial_epochs = 30

Compile the before built model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

Train the model!

In [ ]:
generator = keras.preprocessing.image.ImageDataGenerator()
batch_size = 32

history = model.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=(validation_images, validation_labels), epochs=initial_epochs, callbacks=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001))

In [ ]:
model.evaluate(x = validation_images, y = validation_labels)

# 3. Fine tuning the base model

Freeze all layers except for the uppermost layers of the base model. Layzers will be frozen from ***fine_tune_from*** to ***len(base_model.layers)***.

In [ ]:
base_model.trainable = True
fine_tune_from = len(base_model.layers) - 25

for layer in base_model.layers[:fine_tune_from]:
    layer.trainable = False

# don't train the dense layer
model.layers[len(model.layers) - 1].trainable = False
model.layers[len(model.layers) - 2].trainable = False

model.summary()

Compile the model again but decrease the learning rate in order to avoid overfitting.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/100), loss='categorical_crossentropy', metrics=['accuracy'])

Train the uppermost layers of the base model and proceed where you left off in the initial training by using the last epoch of the first training as the initial epoch for fine tuning.

In [ ]:
model.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=generator.flow(validation_images, validation_labels, batch_size), initial_epoch=history.epoch[-1], epochs=initial_epochs + 25, callbacks=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3))

# 4. Save the model

Info: Loading can be done via: ***model = tf.keras.models.load_model(PATH_TO_MODEL_FILE)***


In [ ]:
# for now until it is implemented properly: manually make sure that the path exists (in this case '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/')
save_path = '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/w100_h40_crop_no_contrast_97_val_99_train.keras'

model.save(save_path)